## Importing Dependencies

In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Technical Indicators Calculation

In [3]:
df = pd.read_csv('10_Year_Historical_Preprocessed.csv')

In [4]:
def exponential_moving_avg(df: pd.DataFrame, price_col='Adj Close', window_size=20):
    return df[price_col].ewm(span=window_size, adjust=False).mean()

def macd_line(df: pd.DataFrame, price_col='Adj Close', short_window=12, long_window=26):
    short_ema = exponential_moving_avg(df, price_col, short_window)
    long_ema = exponential_moving_avg(df, price_col, long_window)
    return short_ema - long_ema

def macd_signal(df: pd.DataFrame, price_col='Adj Close', signal_window=9, short_window=12, long_window=26):
    macd = macd_line(df, price_col, short_window, long_window)
    return macd.ewm(span=signal_window, adjust=False).mean()


In [5]:
df['EMA 20'] = df.groupby('Ticker').apply(lambda x: exponential_moving_avg(x, price_col='Adj Close', window_size=20)).reset_index(level=0, drop=True)
df['MACD Line'] = df.groupby('Ticker').apply(lambda x: macd_line(x, price_col='Adj Close')).reset_index(level=0, drop=True)
df['MACD Signal'] = df.groupby('Ticker').apply(lambda x: macd_signal(x, price_col='Adj Close')).reset_index(level=0, drop=True)

In [6]:
def scale_column(group, col):
    scaler = MinMaxScaler()
    group[col] = scaler.fit_transform(group[col].values.reshape(-1, 1))
    return group

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
columns_to_scale = list(df.columns[2:])

with tqdm(total=len(columns_to_scale), desc='Scaling columns', unit='column') as pbar:
    for col in columns_to_scale:
        df = df.groupby('Ticker').apply(lambda x: scale_column(x, col)).reset_index(drop=True)
        pbar.update(1)


Scaling columns:   0%|          | 0/9 [00:00<?, ?columns /s]

In [11]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')